In [4]:
import pandas as pd
import statsmodels.api as sm
import linearmodels as lm
import pyreadstat
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from scipy.stats import pearsonr
from linearmodels.panel import compare


In [5]:
import sys
sys.path.append('/Applications/Stata/utilities')
from pystata import config
config.init('se')

In [6]:
data_stata, meta = pyreadstat.read_dta(r'/Users/dhruvpandit/Documents/GitHub/ISEG_AEF/Econometrics/Micro/blundbondbalanc_2.dta')
label_dict = {}
for var_name, label in zip(meta.column_names, meta.column_labels):
    label_dict[var_name] = label


In [7]:
data_stata

,id,ly,ln,lk,year,D83,D84,D85,D86,D87,D88,D89
0,886,4.579228,3.480440,3.254327,1982.0,0,0,0,0,0,0,0
1,886,4.472189,3.357781,3.346090,1983.0,1,0,0,0,0,0,0
2,886,4.567035,3.360103,3.485277,1984.0,0,1,0,0,0,0,0
3,886,4.885006,3.315239,3.667849,1985.0,0,0,1,0,0,0,0
4,886,4.999058,3.380052,3.831042,1986.0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3931,989349,7.450092,5.983883,6.155892,1985.0,0,0,1,0,0,0,0
3932,989349,7.576933,5.971417,6.210421,1986.0,0,0,0,1,0,0,0
3933,989349,7.767560,5.923970,6.297313,1987.0,0,0,0,0,1,0,0
3934,989349,7.856476,6.002237,6.334439,1988.0,0,0,0,0,0,1,0


In [8]:
%%stata
use /Users/dhruvpandit/Documents/GitHub/ISEG_AEF/Econometrics/Micro/blundbondbalanc_2.dta
xtset id year, yearly
xtdescribe


. use /Users/dhruvpandit/Documents/GitHub/ISEG_AEF/Econometrics/Micro/blundbond
> balanc_2.dta

. xtset id year, yearly

Panel variable: id (strongly balanced)
 Time variable: year, 1982 to 1989
         Delta: 1 year

. xtdescribe

      id:  886, 1030, ..., 989349                            n =        492
    year:  1982, 1983, ..., 1989                             T =          8
           Delta(year) = 1 year
           Span(year)  = 8 periods
           (id*year uniquely identifies each observation)

Distribution of T_i:   min      5%     25%       50%       75%     95%     max
                         8       8       8         8         8       8       8

     Freq.  Percent    Cum. |  Pattern
 ---------------------------+----------
      492    100.00  100.00 |  11111111
 ---------------------------+----------
      492    100.00         |  XXXXXXXX

. 


In [9]:
%%stata
xtsum ly ln lk


Variable         |      Mean   Std. dev.       Min        Max |    Observations
-----------------+--------------------------------------------+----------------
ly       overall |  5.905447   1.991932   1.134937   11.67105 |     N =    3936
         between |             1.976804   1.408586   11.49127 |     n =     492
         within  |             .2588288   4.189762   7.355696 |     T =       8
                 |                                            |
ln       overall |  4.752687   2.283474  -2.253844   11.23016 |     N =    3936
         between |             2.270097  -.8955042   11.14857 |     n =     492
         within  |             .2647287   3.152142   6.033032 |     T =       8
                 |                                            |
lk       overall |  3.863383   2.081006  -1.295894   10.13477 |     N =    3936
         between |             2.065382  -.3310543   9.906047 |     n =     492
         within  |             .2690222   2.357184   5.506763 |     T =

#### Pooled OLS

In [10]:
%%stata
reg ly l.ly ln l.ln lk l.lk D84-D89, vce(cluster id)
estimates store POLS


. reg ly l.ly ln l.ln lk l.lk D84-D89, vce(cluster id)

Linear regression                               Number of obs     =      3,444
                                                F(11, 491)        =   59731.41
                                                Prob > F          =     0.0000
                                                R-squared         =     0.9942
                                                Root MSE          =     .15155

                                   (Std. err. adjusted for 492 clusters in id)
------------------------------------------------------------------------------
             |               Robust
          ly | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          ly |
         L1. |   .9564256   .0062554   152.90   0.000     .9441351    .9687162
             |
          ln |
         --. |   .4153361    .023799    17.45   0.000     .3685757    .

#### Fixed Effects

In [11]:
%%stata
xtreg ly l.ly ln l.ln lk l.lk D84-D89, fe vce(cluster id)
estimates store FE


. xtreg ly l.ly ln l.ln lk l.lk D84-D89, fe vce(cluster id)

Fixed-effects (within) regression               Number of obs     =      3,444
Group variable: id                              Number of groups  =        492

R-squared:                                      Obs per group:
     Within  = 0.7081                                         min =          7
     Between = 0.9941                                         avg =        7.0
     Overall = 0.9897                                         max =          7

                                                F(11,491)         =     331.59
corr(u_i, Xb) = 0.9276                          Prob > F          =     0.0000

                                   (Std. err. adjusted for 492 clusters in id)
------------------------------------------------------------------------------
             |               Robust
          ly | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+--------------------------------

#### First Differences

In [12]:
%%stata
reg D.(ly l.ly ln l.ln lk l.lk D85-D89), nocons vce(cluster id)
estimates store FD


. reg D.(ly l.ly ln l.ln lk l.lk D85-D89), nocons vce(cluster id)

Linear regression                               Number of obs     =      2,952
                                                F(10, 491)        =      92.65
                                                Prob > F          =     0.0000
                                                R-squared         =     0.3326
                                                Root MSE          =     .14912

                                   (Std. err. adjusted for 492 clusters in id)
------------------------------------------------------------------------------
             |               Robust
        D.ly | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          ly |
         LD. |  -.0481349    .023618    -2.04   0.042    -.0945397   -.0017301
             |
          ln |
         D1. |   .3613453   .0269482    13.41   0.000     .3

In [13]:
%%stata
estimates table POLS FE FD, star(.1 .05 .01)


--------------------------------------------------------------
    Variable |     POLS             FE              FD        
-------------+------------------------------------------------
          ly |
         L1. |  .95642564***    .55043701***                  
         LD. |                                  -.0481349**   
             |
          ln |
         --. |  .41533612***    .38556523***                  
         L1. | -.39156312***   -.22218468***                  
         D1. |                                   .3613453***  
         LD. |                                  .09486793***  
             |
          lk |
         --. |  .27799009***    .42332897***                  
         L1. | -.26537853***   -.35674603***                  
         D1. |                                  .60170257***  
         LD. |                                 -.31780568***  
             |
         D84 |  .10580755***    .09715085***                  
         D85 |  .03740616**

#### Arellano & Bond
- one-step GMM, robust s.e., strictly exogenous variables
- write out variables, have to avoid dummy trap (you lose two observatios in time), state which variables to use for instruments -> put in all variables you know are strictly exogenous (like dummies which are deterministic), gmm includes all variables that will be instrumented, what kind of gmm (one step, two step)
- if strictly exogenous, you do not need instruments
- because using arellano and bond, only estimating first difference so we use noleveleq
- by default uses one step gmm
- calculates robust se by clusters
- designed for microdata, assumed that the units are independent, corrects for within serial correlation and heterosked among units

In [14]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(ln lk D85-D89) gmm(l.ly) noleveleq robust
estimates store AB1_SE


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(ln lk D85-D89) gmm(l.ly) nolevel
> eq robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 28                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
             |               Robust
          ly | Coeffi

- degree of overidentification 18, usually higher than that
- sargan test shows strong overidentificaiton

one-step GMM, robust s.e., predetermined Variables

In [18]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk) noleveleq robust
estimates store AB1_PD


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk) no
> leveleq robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 80                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
             |               Robust
          ly | C

one-step GMM, robust s.e., endogenous variables
restrics instruments used with the lag option

In [20]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk, lag(2 .)) noleveleq robust
estimates store AB1_End


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk, la
> g(2 .)) noleveleq robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 68                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
             |               Robust
      

In [21]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(4 .)) gmm(ln lk, lag(3 .)) noleveleq robust
estimates store AB1_End_2


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(4 .)) gmm
> (ln lk, lag(3 .)) noleveleq robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 41                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
             |               Rob

Two-step GMM, endogenous variables, windmeijer correction

In [22]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(4 .)) gmm(ln lk, lag(3 .)) noleveleq twostep robust
estimates store AB2_End


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(4 .)) gmm
> (ln lk, lag(3 .)) noleveleq twostep robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate optimal weighting matrix for two-ste
> p estimation.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, two-step difference GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      2952
Time variable : year                            Number of groups   =       492
Number of instruments = 41                      Obs per group: min =         6
Wald chi2(0)  =         .                                      avg =      6.00
Prob > chi2   =         .                                      max =         6
------------------------------------------------------------------------------
             | 

#### Blundell & Bond
* one-step GMM, robust s.e., predetermined Variables

In [23]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk) robust
estimates store BB1_PD


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly) gmm(ln lk) ro
> bust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step system GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      3444
Time variable : year                            Number of groups   =       492
Number of instruments = 101                     Obs per group: min =         7
Wald chi2(10) =  3.94e+06                                      avg =      7.00
Prob > chi2   =     0.000                                      max =         7
------------------------------------------------------------------------------
             |               Robust
          ly | Coefficient  st

In [24]:
%%stata
xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(5 .)) gmm(ln lk, lag(4 .)) robust
estimates store BB1_End


. xtabond2 ly l.ly ln l.ln lk l.lk D85-D89, iv(D85-D89) gmm(l.ly, lag(5 .)) gmm
> (ln lk, lag(4 .)) robust
Favoring space over speed. To switch, type or click on mata: mata set matafavor
>  speed, perm.
  Using a generalized inverse to calculate robust weighting matrix for Hansen t
> est.
  Difference-in-Sargan/Hansen statistics may be negative.

Dynamic panel-data estimation, one-step system GMM
------------------------------------------------------------------------------
Group variable: id                              Number of obs      =      3444
Time variable : year                            Number of groups   =       492
Number of instruments = 39                      Obs per group: min =         7
Wald chi2(10) =  2.42e+06                                      avg =      7.00
Prob > chi2   =     0.000                                      max =         7
------------------------------------------------------------------------------
             |               Robust
          